In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240706'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/6.type.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Type No'] = input_['Type No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        for b in range(666):
            try:
                import requests

                import sys
                sys.path.append('../00.Tools')
                from crawler_configuration import get_header, get_proxy

                resp = requests.get(input_.loc[a, 'Type Url'],
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(10, 10)).text

                # = = = = = = = = = = = = = = =

                from bs4 import BeautifulSoup
                from lxml import etree

                soup = BeautifulSoup(resp.strip(), 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                count = int(html.xpath('//span[@id="lblResultCount"]/text()')[0].strip())

                # = = = = = = = = = = = = = = =

                if count <= 0:
                    continue

                # = = = = = = = = = = = = = = =

                import math

                df_temp = pd.DataFrame([{'Type No': input_.loc[a, 'Type No'],
                                         'Type': input_.loc[a, 'Type'],
                                         'Type Url': input_.loc[a, 'Type Url'],
                                         'SKU Count': count,
                                         'Page': math.ceil(count / 100)}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Type No': input_.loc[a, 'Type No'],
                                     'Type': input_.loc[a, 'Type'],
                                     'Type Url': input_.loc[a, 'Type Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'Type No']}.{input_.loc[a, 'Type']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Type No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/7.page-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Type No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./{param_date}/page_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：  1] - 1267.Liftgate Strap Repair Kit
[剩余数量：1000] - [当前时间：16:59:22]

[状态：ok，尝试次数：  1] - 1270.Locking Hub Seal Kit
[剩余数量：999] - [当前时间：16:59:22]

[状态：ok，尝试次数：  1] - 1269.Locking Hub
[剩余数量：998] - [当前时间：16:59:24]

[状态：ok，尝试次数：  1] - 1272.Luggage Compartment Light Bulb
[剩余数量：997] - [当前时间：16:59:24]

[状态：ok，尝试次数：  1] - 1242.License Lamp Lens
[剩余数量：996] - [当前时间：16:59:25]

[状态：ok，尝试次数：  1] - 1261.Liftgate Latch Release Switch Seal
[剩余数量：995] - [当前时间：16:59:26]

[状态：ok，尝试次数：  1] - 1274.Manual Transmission Drain Plug Gasket
[剩余数量：994] - [当前时间：16:59:26]

[状态：ok，尝试次数：  1] - 1277.Manual Transmission Shift Knob
[剩余数量：993] - [当前时间：16:59:27]

[状态：ok，尝试次数：  1] - 1275.Manual Transmission Output Shaft Speed Sensor
[剩余数量：992] - [当前时间：16:59:27]

[状态：ok，尝试次数：  1] - 1266.Liftgate Pull Handle Clip
[剩余数量：991] - [当前时间：16:59:28]

[状态：ok，尝试次数：  1] - 1279.Manual Transmission Shift Tower Kit
[剩余数量：990] - [当前时间：16:59:29]

[状态：ok，尝试次数：  1] - 1260.Liftgate Latch Release Motor Gear Repair Kit
[剩余数量：989] - [当前